In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import re

In [4]:
r = requests.get('https://go.drugbank.com/drugs?utf8=%E2%9C%93&approved=0&approved=1&nutraceutical=0&nutraceutical=1&illicit=0&illicit=1&investigational=0&investigational=1&withdrawn=0&withdrawn=1&experimental=0&experimental=1&us=0&ca=0&eu=0&commit=Apply+Filter')
print(r.status_code)
html = r.content
soup = BeautifulSoup(html, 'html.parser')

200


All the filters are selected on the cell here above to collect the largest dataset

In [5]:
links = soup.find_all('a')
links_db = re.findall('/drugs/DB\d+', str(links))
links_db = pd.DataFrame(links_db, columns = ['DB_numbers'])
len(links_db)

25

As expected, we get 25 results per page

In [6]:
df_db_num = links_db.copy()
df_db_num['DB_numbers'] = links_db['DB_numbers'].str.replace('/drugs/','')
#Idk if i'm gonna use it, but I'd like to keep a list with DB codes aswell

In [7]:
total_results = int(soup.find('div', attrs = {'class':'page_info'}).get_text().split()[-3])
total_results

11804

In [8]:
results_pages = list(range(474))
results_pages = results_pages[1:]
print(f'First page: {results_pages[0]}, last page: {results_pages[-1]}')

First page: 1, last page: 473


### **CAREFUL: THE CODE IN THE CELL HEREUNDER TAKES 4-6 MIN TO RUN**

In [9]:
substances = []

for i in results_pages:
    try:
        r = requests.get(f'https://go.drugbank.com/drugs?approved=1&c=name&ca=0&d=up&eu=0&experimental=1&illicit=1&investigational=1&nutraceutical=1&page={i}&us=0&withdrawn=1')
        soup = BeautifulSoup(r.content, 'html.parser')
        substances += soup.find_all('td', attrs={'class':'name-value text-sm-center drug-name'})
        page_suffix = [substance.find('a').get('href') for substance in substances]
        print(i)
    except:
        print(f'Error on page {i}.')
        continue

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [10]:
len(substances)

11804

In [11]:
page_suffix = [substance.find('a').get('href') for substance in substances]
len(page_suffix)

11804

### **CAREFUL: THE CELL BELOW TAKES 1 HOUR TO RUN. IF IT HAS TE BE MODIFIED FOR A TEST, ONLY RUN ON 500-1000 FIRST RESULTS**

### Let's get AHFS classes:

In [ ]:
AHFS_class = [] #THIS LIST SHOULD BE MY TARGET, BUT SINCE ONLY A FEW COMPOUNDS HAVE IT, IT RAISES TOO MANY ERRORS. I'D LIKE TO REPLACE THE VALUE BY 'NULL' IF IT DOESN'T EXISTS

for item in page_suffix:
    r = requests.get(f'https://go.drugbank.com/{item}')
    html = r.content
    soup = BeautifulSoup(html, 'html.parser')

    try:
      AHFS_class.append(soup.find_all('ul', attrs = {'class':'list-unstyled table-list'})[3].text)
    except:
      AHFS_class.append(None)
      print(f'AHFS_class error on page {item}')
      continue

AHFS_class error on page /drugs/DB14136
AHFS_class error on page /drugs/DB07949
AHFS_class error on page /drugs/DB08664
AHFS_class error on page /drugs/DB03335
AHFS_class error on page /drugs/DB02047
AHFS_class error on page /drugs/DB11344
AHFS_class error on page /drugs/DB02205
AHFS_class error on page /drugs/DB03121
AHFS_class error on page /drugs/DB07409
AHFS_class error on page /drugs/DB06830
AHFS_class error on page /drugs/DB06931
AHFS_class error on page /drugs/DB03062
AHFS_class error on page /drugs/DB14093
AHFS_class error on page /drugs/DB03216
AHFS_class error on page /drugs/DB04746
AHFS_class error on page /drugs/DB04264
AHFS_class error on page /drugs/DB07430
AHFS_class error on page /drugs/DB08747
AHFS_class error on page /drugs/DB07216
AHFS_class error on page /drugs/DB15571
AHFS_class error on page /drugs/DB06888
AHFS_class error on page /drugs/DB07596
AHFS_class error on page /drugs/DB08553
AHFS_class error on page /drugs/DB03455
AHFS_class error on page /drugs/DB02367


In [ ]:
AHFS = pd.DataFrame(AHFS_class)

In [ ]:
AHFS

,0
0,None
1,None
2,None
3,None
4,None
...,...
11799,None
11800,None
11801,None
11802,None


In [ ]:
AHFS[0].unique()

array([None,
       '08:18.08.20 — Nucleoside and Nucleotide Reverse Transcriptase Inhibitors',
       'Baxter International Inc.\nFisher Clinical Services Inc.',
       '10:00.00 — Antineoplastic Agents',
       '28:92.00 — Miscellaneous Central Nervous System Agents',
       '68:20.02 — Alpha-glucosidase Inhibitors',
       '24:24.00 — Beta-adrenergic Blocking Agents',
       '20:12.04.08 — Coumarin Derivatives',
       '28:08.92 — Miscellaneous Analgesics and Antipyretics',
       '52:40.00 — Antiglaucoma Agents\n52:40.12 — Carbonic Anhydrase Inhibitors',
       'Barr laboratories inc\nUsl pharma inc\nWatson laboratories inc\nEli lilly industries inc',
       'Mission pharmacal co',
       'Amend\nAmerican Regent\nBedford Labs\nBen Venue Laboratories Inc.\nBioniche Pharma\nBoehringer Ingelheim Ltd.\nBristol-Myers Squibb Co.\nCardinal Health\nCumberland Pharmaceuticals\nHospira Inc.\nLuitpold Pharmaceuticals Inc.\nRoxane Labs\nSpectrum Pharmaceuticals',
       '28:08.04.24 — Salicyla

In [ ]:
from google.colab import files
AHFS.to_csv('AHFS.csv', sep = ',')